In [83]:
import os, random
import numpy as np
import matplotlib.image as pltimage
import matplotlib.pyplot as plt
from PIL import Image
def show_files(label_dir):
    all_files = []
    file_list = os.listdir(label_dir)
    for file in file_list:
        cur_path = os.path.join(label_dir, file)
        root, extension = os.path.splitext(cur_path)
        ext_lower = extension.lower()
        if ext_lower == ".png" or ext_lower == ".jpg" or ext_lower == ".jpeg":
            all_files.append(cur_path)
    return all_files

def obtainXY(data_dir, shuffle=False):
    X = []
    Y = []
    classes =  [i for i in range(10)]
    for label in classes:
        all_files = show_files(data_dir + str(label))
        for file in all_files:
            img = Image.open(file)
            img = img.resize((28, 28), Image.ANTIALIAS)
            gray = img.convert("L")
            x = np.asarray(gray)/255.
            y = label
            X.append(x)
            Y.append(y)
    X = np.array(X)
    Y = np.array(Y)
    if shuffle:
        indexs = [i for i in range(len(X))] 
        random.shuffle(indexs) #乱序
        X = X[indexs]
        Y = Y[indexs]
    return X, Y

dataset_dir = "./dataset/"
X, Y = obtainXY(dataset_dir, shuffle=True)
print(X.shape, Y.shape)

(129, 28, 28) (129,)


In [84]:
from collections import  Counter

def print_pecent(data, class_num=10):
    y_ = list(data)
    size = len(y_)
    class_counter = Counter(y_)
    print(class_counter, size)
    for i in range(class_num):
        print(i, "占比:", round(class_counter[i]/size, 3))

print_pecent(Y)

Counter({1: 16, 5: 16, 9: 16, 3: 14, 4: 12, 2: 12, 7: 12, 8: 11, 6: 10, 0: 10}) 129
0 占比: 0.078
1 占比: 0.124
2 占比: 0.093
3 占比: 0.109
4 占比: 0.093
5 占比: 0.124
6 占比: 0.078
7 占比: 0.093
8 占比: 0.085
9 占比: 0.124


In [85]:
import tensorflow as tf
from keras import layers, regularizers
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

def creat_nn(print_summary=False):
    model = Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    if print_summary:
        model.summary()
    return model

def train_cnn(train_x, train_y, batch_size, epochs, load_save_model=False):
    model_path = './model/'
    if load_save_model:
        print('train load save model path', model_path)
        model = load_model(model_path)
    else:
        print('train create new model')
        model = creat_nn(print_summary=True)
    model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, verbose=1)
    model.save(model_path)
    
    #save tflite model
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    tflite_path = model_path + 'mymodel.tflite'
    save = open(tflite_path, 'wb')
    save.write(tflite_model)
    save.close()
    return model

train_x = X
# 将label转成向量，如 2 => [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
train_y = tf.keras.utils.to_categorical(Y, num_classes=10)
# input output shape
print(train_x.shape, train_y.shape)
# first train model
model = train_cnn(train_x, train_y, batch_size=16, epochs=12, load_save_model=False)

(129, 28, 28) (129, 10)
train create new model
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 784)               0         
                                                                 
 dense_14 (Dense)            (None, 128)               100480    
                                                                 
 dense_15 (Dense)            (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
Epoch 1/12
9/9 [==============================] - 0s 2ms/step - loss: 2.2839 - accuracy: 0.1008
Epoch 2/12
9/9 [==============================] - 0s 1ms/step - loss: 1.8984 - accuracy: 0.5194
Epoch 3/12
9/9 [==============================] - 0s 1ms/step - loss: 1.6

INFO:tensorflow:Assets written to: ./model/assets


INFO:tensorflow:Assets written to: /tmp/tmpvvfe3_ys/assets


INFO:tensorflow:Assets written to: /tmp/tmpvvfe3_ys/assets


In [90]:
def verify_model(model, valid_x, valid_y, num_classes=10):
    y_pred = model.predict(valid_x)
    y_pred = np.argmax(y_pred, axis=1)
    size = [0 for i in range(num_classes)]
    correct = [0 for i in range(num_classes)]
    for i in range(len(y_pred)):
        label = valid_y[i]
        pred = y_pred[i]
        size[label] += 1
        if pred == label:
            correct[label] += 1
    print("total accuracy:", sum(correct)/sum(size))
    for i in range(num_classes):
        print("label:", str(i), " accuracy:", correct[i]/max(size[i], 1))
    
verify_model(model, X, Y)

5/5 [==============================] - 0s 1ms/step
accuracy 0.9844961240310077
[10, 16, 12, 14, 12, 16, 10, 12, 11, 16] [10, 16, 12, 14, 12, 16, 9, 11, 11, 16]


In [80]:
# retrain model
train_cnn(train_x, train_y, batch_size=16, epochs=8, load_save_model=True)

train load save model path ./model/
Epoch 1/8
9/9 [==============================] - 0s 2ms/step - loss: 0.3483 - accuracy: 0.9690
Epoch 2/8
9/9 [==============================] - 0s 2ms/step - loss: 0.2401 - accuracy: 1.0000
Epoch 3/8
9/9 [==============================] - 0s 2ms/step - loss: 0.1915 - accuracy: 1.0000
Epoch 4/8
9/9 [==============================] - 0s 2ms/step - loss: 0.1488 - accuracy: 1.0000
Epoch 5/8
9/9 [==============================] - 0s 2ms/step - loss: 0.1200 - accuracy: 1.0000
Epoch 6/8
9/9 [==============================] - 0s 2ms/step - loss: 0.0982 - accuracy: 1.0000
Epoch 7/8
9/9 [==============================] - 0s 2ms/step - loss: 0.0837 - accuracy: 1.0000
Epoch 8/8
9/9 [==============================] - 0s 2ms/step - loss: 0.0707 - accuracy: 1.0000
INFO:tensorflow:Assets written to: ./model/assets


INFO:tensorflow:Assets written to: ./model/assets


INFO:tensorflow:Assets written to: /tmp/tmpssbbe_4w/assets


INFO:tensorflow:Assets written to: /tmp/tmpssbbe_4w/assets
